In [1]:
import pandas as pd
import os
from langchain_openai import AzureChatOpenAI
import json
from langchain_core.prompts import ChatPromptTemplate
from langsmith import traceable
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
AZURE_OPENAI_API_KEY=os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT=os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_VERSION=os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    deployment_name="gpt-4o-mini",
    temperature=0.1
)


In [4]:
from pydantic import BaseModel, Field
from typing import TypedDict, List

class Plan(BaseModel):
    goal: str = Field(description="User goal")
    steps: List[str] = Field(description="Ordered steps to solve goal")

class AgentState(TypedDict):
    goal: str
    steps: List[str]
    current_step: int
    step_results: List[str]
    final_answer: str

In [5]:
@traceable(name="Planner Node")
def planner_node(state: AgentState):
    planner_prompt = f"""
        You are a planning agent.

        Break the user's goal into clear ordered steps. Steps should not be more than 3.

        Goal: {state["goal"]}
        """
    plan = llm.with_structured_output(Plan).invoke(planner_prompt)
    return {
        "steps": plan.steps,
        "current_step": 0,
        "step_results": []
    }


In [6]:
@traceable(name="Executor Node")
def executor_node(state: AgentState):
    step = state["steps"][state["current_step"]]

    executor_prompt = f"""
        Execute the following step in context of the overall goal.

        Goal: {state["goal"]}
        Current Step: {step}

        Provide result for this step within 400 words.
        """

    response = llm.invoke(executor_prompt)

    return {
        "step_results": state["step_results"] + [response.content],
        "current_step": state["current_step"] + 1
    }


In [7]:
@traceable(name="Continue Executing Node")
def should_continue(state: AgentState):
    if state["current_step"] < len(state["steps"]):
        return "executor"
    else:
        return "final"


In [8]:
@traceable(name="Final Node")
def final_node(state: AgentState):
    return {
        "final_answer": "\n\n".join(state["step_results"])
    }


In [9]:
from langgraph.graph import StateGraph, END

builder = StateGraph(AgentState)

builder.add_node("planner", planner_node)
builder.add_node("executor", executor_node)
builder.add_node("final", final_node)

builder.set_entry_point("planner")

builder.add_edge("planner", "executor")
builder.add_conditional_edges("executor", should_continue)
builder.add_edge("final", END)

graph = builder.compile()


In [10]:
result = graph.invoke({
    "goal": "Explain how to design a production-grade RAG system"
})


In [12]:
result

{'goal': 'Explain how to design a production-grade RAG system',
 'steps': ['Define the requirements and objectives of the RAG system, including data sources, user needs, and performance metrics.',
  'Select appropriate technologies and frameworks for building the RAG system, considering scalability, reliability, and integration capabilities.',
  'Implement the system architecture, focusing on data processing, model training, and deployment, followed by thorough testing and optimization for production use.'],
 'current_step': 3,
 'step_results': ["### Defining Requirements and Objectives for a Production-Grade RAG System\n\nTo design a production-grade Retrieval-Augmented Generation (RAG) system, it is crucial to clearly define its requirements and objectives. This foundational step ensures that the system meets user needs, leverages appropriate data sources, and achieves desired performance metrics.\n\n#### 1. **Data Sources**\n\nThe RAG system should integrate multiple data sources to

In [11]:
print(result["final_answer"])

### Defining Requirements and Objectives for a Production-Grade RAG System

To design a production-grade Retrieval-Augmented Generation (RAG) system, it is crucial to clearly define its requirements and objectives. This foundational step ensures that the system meets user needs, leverages appropriate data sources, and achieves desired performance metrics.

#### 1. **Data Sources**

The RAG system should integrate multiple data sources to enhance its retrieval capabilities. Key data sources may include:

- **Structured Data**: Databases containing factual information, such as product catalogs, FAQs, or knowledge bases.
- **Unstructured Data**: Textual data from documents, articles, and web pages that provide context and depth to the responses.
- **APIs**: External services that can provide real-time data, such as news feeds, weather updates, or social media content.
- **User-Generated Content**: Feedback, reviews, and queries from users that can help refine the system's understanding of